<a href="https://colab.research.google.com/github/pleunipennings/GOLD_EDSS/blob/main/Module_AI_ML_ImageClassification/FrogEmbryos_TransferLearningDemo_EDSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Transfer Learning for Biomedical Images

*Note: this notebook was created by Dr Ilmi Yoon (Computer Science, SFSU).* 

*Important: if you want to make changes to this notebook and save these changes, you need to save a copy of the notebook in your own google drive.*

In biology and medicine, the task of predicting a condition from image data is important to diagnose a variety of medical conditions, to identify organisms and to do many other things. In computer science, this task is called **image classification**. Algorithmic image classification is usually performed using a specific type of algorithm called **deep learning**. Deep learning is a subset of machine learning. 

In image classification using deep learning, the workflow usually goes as follows:

1. Create a dataset with a folder for each class of image (e.g., dogs in one folder, cats in the other).
2. Split this dataset into training, validation, and testing sets.
3. Train the model using the training set.
4. Optimize the training parameters using the validation set.
5. Evaluate the model using the testing set.


Deep learning has been successfully used to accurately classify many types of biomedical images. Recently, a classification algorithm was able to detect breast cancer from mammography images with an accuracy of 98% [[1]](https://www.nature.com/articles/s41598-019-48995-4). In collaboration with Dr Ilmi Yoon, Domingo Lab and her graduate students, the accuracy of the classification of images of tadpole (frog) embryos stained with muscle and intersomitic boundaries was improved using deep transfer learning. These images were classified into two categories: control and mutant. This notebook outlines the core algorithm used for this task in a general way, so that future experiments with novel datasets can easily be conducted.



## Background

Before any code is run, it is important to understand some basics of neural networks and transfer learning.


### Deep Neural Networks
A deep neural network is a conditional probability estimator which can predict the probability of some condition given some input. This makes it useful for image classification, which involves detecting the probability of a condition when provided with an image. This project uses an industry standard neural network architecture called VGG16 (see image), which accepts color images as input and predicts the probability that the image belongs to any of 1000 general image classes. 

![VGG16 Architecture](https://miro.medium.com/max/470/1*3-TqqkRQ4rWLOMX-gvkYwA.png)


This network is composed of many network layers. Each layer further processes the image. The output layer is the orange block at the end of the network, which outputs the probability distribution among the image classes.


### Transfer Learning
While Deep Learning has advanced recognition of patterns and understanding of data in various formats, recent advancements in Convolution Neural Network (CNN) achieve super-human level computer vision in natural photo images [3]. Typical human error in image recognition is 5% while the latest CNN models such as VGG19 achieves 3% or lower rates [4]. 

These achievements are made with advanced Convolution Neural Network (CNN) architecture design, careful selection of hyperparameters, a very large data set (more than 15 million labeled images) and high computation power for long deep learning training; a typical computer science lab will have difficulty in achieving the same results. Fortunately, to promote the advancement of deep learning applications, Google, the Oxford research group or other groups made their fully trained CNN publicly available through “Transfer Learning” technology [4]. 

Although these fully trained CNN are trained with objects naturally found from photo images such as dog, cat, car, etc., scientists or domain experts have started to use these fully-trained-CNN to classify their scientific data with small additional training process (called fine-tuning).

Transfer learning is the process fine-tuning a well trained neural network (like VGG16) for a specific task. This has been proven to be effective at classifying medical image data with minimal training time and smaller datasets than would otherwise be required for high levels of accuracy. The output layer  with 1000 classes is replaced with a custom layer with the number of outputs equal to the number of classes of image we have. In this project, the classes are control and mutant. The network is re-trained with a small hand-labeled dataset.

#Getting started 

### Configuring the Virtual Interpreter
The colab environment runs code cells on a virtual cloud computer owned by Google. To ensure that the code is executed efficiently, GPU (graphics processing) must be enabled:

* Navigate to 'Runtime' > 'Change Runtime Type'
* Set 'Hardware Accelerator' to 'GPU'

Once you have performed this step, the output of the code cell below should read 'GPU 0' followed by the name of the graphics processor.

In [ ]:
!nvidia-smi --list-gpus

### Importing the Software Libraries

The following code imports some software libraries necessary to execute the project. We use Numpy, Keras (which is now part of tensorflow), scikit-learn, matplotlib and itertools. 

In [ ]:
%tensorflow_version 2.x
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib.image as mpimg
print("Import Success!")

## 1 - Getting the Data

The data must be loaded into the Keras API (a software library) so that it can be easily fed to the machine learning model.


### Downloading the Data
First, the data must be downloaded to the virtual machine. With this line of code, you are getting the data from my (Pleuni's) google Drive. 

In [ ]:
!gdown --id '1hC_WFptSb_4JSqJG0OAIYyRonpmaslrh' 

### Extract the Data

Since the images are in a .zip file, they must first be extracted to .jpg images.


In [ ]:
!unzip cropdata.zip
print("Done.")

## 2 - Splitting the Dataset

The data is organised in three folders: train, validation, and test. The training data will be used to train the model. The validation data will be run through the model occationally to ensure that it is generalizing well to input outside the training cases (not overfitting). The validation data is used to determine the optimal amount of training. The test data will not be used until training is completely done in order to measure our peformance. 

In [ ]:
train_path = 'cropdata/train'
valid_path = 'cropdata/valid'
test_path = 'cropdata/test'
print("Paths Loaded.")

### Creating the Data Generators

The model requires the image to be provided as batches of image vectors. By batch, we just mean a consistent number of images. A useful software interface for feeding data into our model is an ImageDataGenerator from the Keras library.  After specifying the data source and other parameters, it will continually generate data batches of specified size for our model to use.

In [ ]:
image_size = (224, 224)
classes = ['control', 'mutant']
print("Training:")
train_batches = ImageDataGenerator() \
                  .flow_from_directory(train_path,
                                       image_size,
                                       classes=classes,
                                       batch_size=15)
print("Validation:")
valid_batches = ImageDataGenerator() \
                  .flow_from_directory(valid_path,
                                       image_size,
                                       classes=classes,
                                       batch_size=5)
print("Testing:")
test_batches = ImageDataGenerator() \
                .flow_from_directory(test_path,
                                     image_size,
                                     classes=classes,
                                     batch_size=64,
                                     shuffle=False)

### Visualizing the Data

The following code will display some of the training images so we can see what they look like. 

In [ ]:
# plots images with labels
def plots(ims, figsize=(20,20), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

imgs, labels = next(train_batches)

plots(imgs[0:5], titles=labels)

## 3 - Creating the Model

In Keras, the deep learning model is represented by layers of processing. 
After importing a pretrained model, a new model is created using this model. All of these layers are copied from the pretrained model to the new model. However, the final output layer will be replaced and retrained using the labeled data, esssentially customizing the output for this specific classification task.

### Loading a Pretrained Model

A pretrained VGG16 model is downloaded using the Keras library. 

In [ ]:
vgg16_model = tensorflow.keras.applications.VGG16()

In [ ]:
vgg16_model.summary()

### Creating a Custom Output Model#

A new model is created copying all layers except the output layer. Notice that these layers are set to be non-trainable. Finally, a custom output layer is added to the model.  This output layer should have the same number of outputs as the number of different classes of data. The softmax activation function conditions the output to be a probability distribuiton adding up to unity. 
In mathematics, the softmax function, also known as softargmax or normalized exponential function, is a function that takes as input a vector of K real numbers, and normalizes it into a probability distribution consisting of K probabilities proportional to the exponentials of the input numbers.
Therefore, our model will output a probability distribution among the classes as output for each input image.

In [ ]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)
for layer in model.layers:
    layer.trainable = False

model.add(Dense(len(classes), activation='softmax'))
model.summary()

### Compiling the Model
In preperation for training, the model is compiled. In order to train it, we must evaluate its performance using a loss function. This function is designed to increase as performance decreases. Therefore, in training, we are essentially minimizing the value of the loss function. Since we are predicting a condition (yes/no), we are using cross-entropy, which is a concept from information theory. If we were instead predicting a numerical value, the correct loss function would be 'mean_squared_error'.

In [ ]:
model.compile(Adam(lr=.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print("Done")

## 4 - Training the model by minimizing Loss

The model is now trained using the backpropagation algorithm to minimize the loss function. The training data will be passed into the model and evaluated using the loss function. Then, the model will be adjusted to decrease the loss.

Here we specifiy the epochs, which is the number of times we pass through the entire training set. These specifications are determined by experimentation with the goal of minimizing validation loss. If the epochs are too high, the model has overfit to the data (too specific). If the epochs are too low, the model has underfit to the data (too general). The validation loss must be compared to training loss to determine if the model is optimally fit to the data.


In [ ]:
model.fit(
     train_batches,
     steps_per_epoch = train_batches.samples // 15,
     validation_data = valid_batches, 
     validation_steps = valid_batches.samples // 5,
     epochs=10, 
     verbose=1)
print("Trining Complete!")

## 5 - Testing the Model

To perform a final test of the model, the test data is fed through the model.  It is important to use data what was removed from the training set so we can test the performance accurately. The accuracy (percentage of images classified correctly) is calculated and a confusion matrix is generated. 

### Test Accuracy

To calculate accuracy, the model is evaluated using the test generator specified earlier.

In [ ]:
test_batches.samples #print the number of samples in the test dataset

In [ ]:
test_batches = ImageDataGenerator() \
                .flow_from_directory(test_path,
                                     image_size,
                                     classes=classes,
                                     batch_size=189, 
                                     shuffle=False) # all images in the test dataset
test_loss, test_acc = model.evaluate(test_batches,
                                     steps=3,
                                     verbose=1) 
print(test_loss)
print(test_acc) 

### Confusion Matrix

A confusion matrix gives more insight than just accuracy: It indicates which images were classified correctly, and for the incorrect ones, it indicates how (mutant instead of control vs. control instead of mutant). It allows us to determine which classes the model performed well on and which classes the model performed poorly on. If a model performs poorly on a specific class, it may be cauesed by an unclean dataset or insufficient quanitity of images of that class.

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
test_batches.reset()
test_imgs, test_labels = next(test_batches)
test_labels = test_labels[:,0]
predictions = model.predict(test_batches, steps=1, verbose=0)
print(np.round(predictions[:,0]))
cm = confusion_matrix(test_labels, np.round(predictions[:,0]))

cm_plot_labels = ['control','mutant']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')


### Look at the misclassified images. 

In [ ]:
labels_pred= np.round(predictions[:,0])
falsenegatives = np.where(np.logical_and(test_labels == 1, labels_pred == 0)) 

In [ ]:
falsenegatives[0][0:5]

In [ ]:
#print(test_labels[0:5])
imgs, labels = next(test_batches)

plots(imgs[falsenegatives[0][0:5]], titles=labels)

## Preparing for reusing the model later 

### Saving the fine-tuned VGG16 model

Now that the model has been trained and tested, it should be saved so we can classify new images without retraining.

In [ ]:
model.save('fineTunedVGG16.h5')
print("Model Saved.")

### Loading the fine-tuned VGG16 model

In a production environment, the model should be loaded without repeating the training process.

In [ ]:
model.load_weights('fineTunedVGG16.h5')
print("Model Loaded.")

## Conclusion

In this notebook, we transfer learning for image classification. First, the data is uploaded and connected to the Keras API. Then, the model is defined by copying a pretrained model and adding a new output layer. Once defined, the model can be trained using the appropriate loss function. Finally, the model is evaluated using a test dataset. With minimal modification, the model can be used to predict more than two classes and scalar values. It is encouraged to use this notebook as a templete for similar models using novel datasets. 